In [7]:
import json
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

# --- CONFIGURATION ---
# Adjust these paths to your project structure
RAW_DATA_DIR = Path("data/raw")
PROCESSED_DATA_DIR = Path("data/processed_yolo")
IMAGES_DIR = PROCESSED_DATA_DIR / "images"
LABELS_DIR = PROCESSED_DATA_DIR / "labels"
SOLAFUNE_JSON_PATH = RAW_DATA_DIR / "train_annotations.json" # Assumed name

# --- MAIN SCRIPT ---

def parse_solafune_json(json_path, raw_image_dir):
    """
    Parses the proprietary Solafune JSON file.
    This version is adapted for the user-provided JSON structure where
    annotations are nested within each image object.
    
    Args:
        json_path (Path): Path to the Solafune annotation JSON file.
        raw_image_dir (Path): Path to the directory containing the raw images.

    Returns:
        dict: A dictionary mapping image filenames to their annotations.
    """
    annotations = {}
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Iterate through the list of image objects in the JSON
    for image_data in data.get('images',):
        filename = image_data['file_name']
        
        if filename not in annotations:
            annotations[filename] = {
                'size': (image_data['width'], image_data['height']),
                'polygons': [] 
            }
        
        # Iterate through the annotations nested within this image
        for ann in image_data.get('annotations',):
            class_name = ann.get('class', 'unknown')
            flat_points = ann['segmentation']
            points = list(zip(flat_points[::2], flat_points[1::2]))

            annotations[filename]['polygons'].append({
                'class': class_name,
                'points': points
            })
            
    return annotations

def convert_to_yolo_format(all_annotations, class_map):
    """
    Converts the parsed annotations to YOLO segmentation format and writes.txt files.
    """
    print("Converting annotations to YOLO format...")
    LABELS_DIR.mkdir(parents=True, exist_ok=True)

    for filename, data in all_annotations.items():
        img_w, img_h = data['size']
        yolo_lines = []

        for polygon in data['polygons']:
            class_name = polygon['class']
            if class_name not in class_map:
                continue
            
            class_index = class_map[class_name]
            
            # Normalize points and flatten the list
            normalized_points = []
            for x, y in polygon['points']:
                norm_x = x / img_w
                norm_y = y / img_h
                normalized_points.extend([f"{norm_x:.6f}", f"{norm_y:.6f}"])
            
            if normalized_points:
                line = f"{class_index} " + " ".join(normalized_points)
                yolo_lines.append(line)

        # Write the.txt file for the image
        label_path = LABELS_DIR / f"{Path(filename).stem}.txt"
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_lines))

def create_dataset_yaml(output_dir, class_names):
    """
    Creates the dataset.yaml file required for YOLO training.
    """
    output_dir = output_dir.resolve()  # convert to absolute path
    content = f"""
train: {output_dir}/images/train/
val: {output_dir}/images/val/

# Classes
nc: {len(class_names)}
names: {class_names}
"""
    with open(output_dir / "dataset.yaml", 'w') as f:
        f.write(content)
    print(f"Created {output_dir / 'dataset.yaml'}")

def copy_files(file_list, split):
    for filename in file_list:
        # Paths
        src_image = RAW_DATA_DIR / "train_images" / filename
        dst_image = IMAGES_DIR / split / filename

        label_filename = f"{Path(filename).stem}.txt"
        src_label = LABELS_DIR / label_filename
        dst_label = LABELS_DIR / split / label_filename

        # Copy image if it exists
        if src_image.exists():
            shutil.copy(src_image, dst_image)
        else:
            print(f"Warning: Image {src_image} does not exist, skipping.")

        # Copy label if it exists
        if src_label.exists():
            shutil.copy(src_label, dst_label)
        else:
            print(f"Warning: Label {src_label} does not exist, skipping.")

def main():
    # --- 1. Setup Directories ---
    shutil.rmtree(PROCESSED_DATA_DIR, ignore_errors=True)
    for split in ['train', 'val']:
        (IMAGES_DIR / split).mkdir(parents=True, exist_ok=True)
        (LABELS_DIR / split).mkdir(parents=True, exist_ok=True)

    # --- 2. Parse Annotations ---
    # This assumes your raw images are in a subdirectory like 'data/raw/images'
    all_annotations = parse_solafune_json(SOLAFUNE_JSON_PATH, RAW_DATA_DIR / "images")
    if not all_annotations:
        print("Parsing failed. Please check the `parse_solafune_json` function and JSON path.")
        return

    class_map = {
    'group_of_trees': 0,
    'individual_tree': 1
}
    class_names = list(class_map.keys())

    # --- 3. Convert to YOLO format (temporary) ---
    convert_to_yolo_format(all_annotations, class_map)

    # --- 4. Split Data and Organize Files ---
    print("Splitting data into train/validation sets...")
    image_files = list(all_annotations.keys())
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    copy_files(train_files, 'train')
    copy_files(val_files, 'val')

    # --- 5. Create YAML file ---
    create_dataset_yaml(PROCESSED_DATA_DIR, class_names)
    
    print("\nData preparation complete.")
    print(f"Train images: {len(train_files)}, Val images: {len(val_files)}")

if __name__ == "__main__":
    main()

Converting annotations to YOLO format...
Splitting data into train/validation sets...
Created C:\Users\hmanasi1\Documents\ADML\Project\data\processed_yolo\dataset.yaml

Data preparation complete.
Train images: 120, Val images: 30


In [8]:
from ultralytics import YOLO
import torch

# --- CONFIGURATION ---
DATASET_YAML = "data/processed_yolo/dataset.yaml"
EPOCHS = 100
IMAGE_SIZE = 640
BATCH_SIZE = 8
MODEL_NAME = 'yolov8s-seg.pt' # s=small, n=nano, m=medium, l=large, x=xlarge

def train():
    # Check for GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load a pretrained YOLOv8 instance segmentation model
    model = YOLO(MODEL_NAME)

    # Train the model
    print("Starting model training...")
    results = model.train(
        data=DATASET_YAML,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        device=device,
        project="runs/train", # Directory to save results
        name="tree_canopy_experiment_1" # Sub-directory name for this run
    )
    print("Training complete.")
    print(f"Best model saved at: {results.save_dir}")

if __name__ == '__main__':
    train()

Using device: cuda
Starting model training...
Ultralytics 8.3.217  Python-3.11.13 torch-2.0.1+cu117 CUDA:0 (NVIDIA RTX A2000 12GB, 12281MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/processed_yolo/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=tree_canopy_experiment_12, nbs=64, nms=False, opset=None, optimize=False, o

In [9]:
from ultralytics import YOLO
from pathlib import Path

# --- CONFIGURATION ---
# Path to the best model weights from your training run
MODEL_WEIGHTS_PATH = "runs/train/tree_canopy_experiment_1/weights/best.pt" 
# Path to the directory with test images
TEST_IMAGES_DIR = "data/test/evaluation_images" 
# Directory to save prediction results
PREDICTION_SAVE_DIR = "runs/predict/submission_run"

def predict():
    # Load the trained model
    model = YOLO(MODEL_WEIGHTS_PATH)

    # Run inference on the test images
    print(f"Running prediction on images in: {TEST_IMAGES_DIR}")
    results = model.predict(
        source=TEST_IMAGES_DIR,
        save=True,          # Save images with predictions
        save_txt=True,      # Save results in.txt files (YOLO format)
        conf=0.25,          # Confidence threshold for detection
        project="runs/predict",
        name="submission_run"
    )
    print(f"Predictions saved to: {PREDICTION_SAVE_DIR}")
    return results

if __name__ == '__main__':
    predict()

Running prediction on images in: data/test/evaluation_images

image 1/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_1.tif: 640x640 6 group_of_treess, 105.5ms
image 2/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_10.tif: 640x640 13 group_of_treess, 12.8ms
image 3/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_11.tif: 640x640 7 group_of_treess, 666.3ms
image 4/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_12.tif: 640x640 18 group_of_treess, 60.7ms
image 5/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_13.tif: 640x640 (no detections), 52.3ms
image 6/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_14.tif: 640x640 4 group_of_treess, 308.1ms
image 7/150 C:\Users\hmanasi1\Documents\ADML\Project\data\test\evaluation_images\10cm_evaluation_15.ti

In [12]:
import json
from pathlib import Path
from PIL import Image
import numpy as np
import cv2

# --- CONFIGURATION ---
PREDICTION_LABELS_DIR = Path("runs/predict/submission_run/labels")  # YOLO .txt output
TEST_IMAGES_DIR = Path("data/test/evaluation_images")               # Original images
SUBMISSION_FILE_PATH = Path("data/test/evaluation_images/jsons/yolo_submission.json")

# Map YOLO class indices to names (must match your training class_map)
CLASS_MAP = {
    0: "group_of_trees",
    1: "individual_tree"
}

DEBUG = True  # Set False to disable prints

def create_submission_file():
    submission_data = {"images": []}

    print(f"Reading predictions from: {PREDICTION_LABELS_DIR}")
    
    for label_file in PREDICTION_LABELS_DIR.glob("*.txt"):
        image_name = f"{label_file.stem}.tif"  # adjust extension if needed
        image_path = TEST_IMAGES_DIR / image_name
        
        if not image_path.exists():
            print(f"Warning: Image {image_name} not found in {TEST_IMAGES_DIR}, skipping.")
            continue

        with Image.open(image_path) as img:
            img_w, img_h = img.size

        annotations = []
        with open(label_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 3:  # class + at least 1 point
                continue
            class_id = int(parts[0])
            norm_coords = [float(p) for p in parts[1:]]
            coords = []
            for i in range(0, len(norm_coords), 2):
                x = norm_coords[i] * img_w
                y = norm_coords[i+1] * img_h
                coords.append([x, y])
            coords = np.array(coords, dtype=np.float32)

            if len(coords) < 3:
                continue  # skip too small polygons

            # Compute minimum bounding rectangle (4 points)
            rect = cv2.minAreaRect(coords)
            box_pts = cv2.boxPoints(rect)  # 4 points
            box_pts = box_pts.flatten().tolist()  # 8 numbers

            annotations.append({
                "class": CLASS_MAP.get(class_id, "unknown"),
                "confidence_score": 1.0,  # YOLOv8-seg does not save confidence in .txt
                "segmentation": [float(round(c)) for c in box_pts]
            })

        submission_data["images"].append({
            "file_name": image_name,
            "width": img_w,
            "height": img_h,
            "cm_resolution": 10,   # adjust if needed
            "scene_type": "unknown",
            "annotations": annotations
        })

        if DEBUG:
            print(f"{image_name}: {len(annotations)} objects processed")

    # Ensure output directory exists
    SUBMISSION_FILE_PATH.parent.mkdir(parents=True, exist_ok=True)
    with open(SUBMISSION_FILE_PATH, 'w') as f:
        json.dump(submission_data, f, indent=4)

    print(f"\nSubmission file created at: {SUBMISSION_FILE_PATH}")
    print(f"Total images processed: {len(submission_data['images'])}")

if __name__ == '__main__':
    create_submission_file()


Reading predictions from: runs\predict\submission_run\labels
10cm_evaluation_1.tif: 6 objects processed
10cm_evaluation_10.tif: 13 objects processed
10cm_evaluation_11.tif: 7 objects processed
10cm_evaluation_12.tif: 18 objects processed
10cm_evaluation_14.tif: 4 objects processed
10cm_evaluation_16.tif: 2 objects processed
10cm_evaluation_17.tif: 14 objects processed
10cm_evaluation_18.tif: 3 objects processed
10cm_evaluation_2.tif: 1 objects processed
10cm_evaluation_21.tif: 1 objects processed
10cm_evaluation_22.tif: 3 objects processed
10cm_evaluation_23.tif: 3 objects processed
10cm_evaluation_25.tif: 1 objects processed
10cm_evaluation_26.tif: 2 objects processed
10cm_evaluation_28.tif: 2 objects processed
10cm_evaluation_29.tif: 6 objects processed
10cm_evaluation_3.tif: 1 objects processed
10cm_evaluation_30.tif: 9 objects processed
10cm_evaluation_31.tif: 3 objects processed
10cm_evaluation_32.tif: 16 objects processed
10cm_evaluation_33.tif: 1 objects processed
10cm_evaluatio

In [15]:
# Notebook: yolo_train_image_prediction.ipynb
import json
from pathlib import Path
from PIL import Image
from shapely.geometry import Polygon
import numpy as np
import cv2
from ultralytics import YOLO

# --- CONFIG ---
TRAIN_IMAGE = Path("data/raw/train_images/10cm_train_3.tif")
OUTPUT_JSON_PATH = Path("data/raw/train_images/jsons/yolo_train_image_predictions.json")
YOLO_MODEL_PATH = "runs/train/tree_canopy_experiment_1/weights/best.pt"
CONFIDENCE_THRESHOLD = 0.5
CLASSES = ["group_of_trees", "individual_tree"]  # match your YOLO training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------
# Helper: Convert mask to polygons
# ------------------------------
def mask_to_polygons(mask, min_area=10):
    polygons_with_scores = []
    mask_uint8 = (mask > 0.5).astype(np.uint8) * 255
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) >= min_area:
            poly = Polygon(cnt[:,0,:])
            polygons_with_scores.append((poly, 1.0))  # YOLO mask has implicit confidence
    return polygons_with_scores

# ------------------------------
# Load YOLO model
# ------------------------------
model_yolo = YOLO(YOLO_MODEL_PATH)

# ------------------------------
# Run prediction
# ------------------------------
results = model_yolo.predict(
    source=str(TRAIN_IMAGE),
    conf=CONFIDENCE_THRESHOLD,
    save=False,
    save_txt=False,
    device=DEVICE
)

yolo_annotations = []
for r in results:
    # r.boxes.cls contains class IDs for each detected object
    class_ids = r.boxes.cls.cpu().numpy()
    
    if hasattr(r, "masks") and r.masks is not None:
        masks = r.masks.data.cpu().numpy()  # [N, H, W]
        for i, mask in enumerate(masks):
            class_name = CLASSES[int(class_ids[i])]
            polys = mask_to_polygons(mask)
            for poly, score in polys:
                coords = np.array(poly.exterior.coords).flatten().tolist()
                yolo_annotations.append({
                    "class": class_name,
                    "confidence_score": float(score),
                    "segmentation": [float(c) for c in coords]
                })


# ------------------------------
# Save JSON
# ------------------------------
image_np = np.array(Image.open(TRAIN_IMAGE))
output_json = {
    "file_name": TRAIN_IMAGE.name,
    "width": image_np.shape[1],
    "height": image_np.shape[0],
    "cm_resolution": 10,
    "scene_type": "unknown",
    "annotations_yolo": yolo_annotations
}

OUTPUT_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(OUTPUT_JSON_PATH, 'w') as f:
    json.dump(output_json, f, indent=4)

print(f"YOLO prediction saved to: {OUTPUT_JSON_PATH}")
print(f"YOLO polygons: {len(yolo_annotations)}")



image 1/1 C:\Users\hmanasi1\Documents\ADML\Project\data\raw\train_images\10cm_train_3.tif: 640x640 2 group_of_treess, 12.5ms
Speed: 9.0ms preprocess, 12.5ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)
YOLO prediction saved to: data\raw\train_images\jsons\yolo_train_image_predictions.json
YOLO polygons: 2
